In [31]:
import os

In [32]:
%pwd

'/Users/akashkumarsinha/Desktop/Text_summerization'

In [33]:
os.chdir("..")

In [34]:
%pwd

'/Users/akashkumarsinha/Desktop'

In [36]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    dataset_download_url: str
    raw_data_dir: Path
    local_data_file: Path
    unzipped_data_dir: Path
    

In [37]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories


In [38]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([Path(self.config.artifacts_root)])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([Path(config.root_dir)])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            dataset_download_url=config.dataset_download_url,
            raw_data_dir=Path(config.raw_data_dir),
            local_data_file=Path(config.local_data_file),
            unzipped_data_dir=Path(config.unzipped_data_dir),
        )

        return data_ingestion_config
        
    

In [39]:
import os
import urllib.request as request
import zipfile
from TextSummarizer.logging import logger
from TextSummarizer.utils.common import get_size



In [40]:
class dataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self) -> str:
        if not os.path.exists(self.config.local_data_file):
            logger.info("Downloading file...")
            filename, headers = request.urlretrieve(
                self.config.dataset_download_url, self.config.local_data_file
            )
            logger.info(
                f"File downloaded successfully! File location: {filename}, size: {get_size(self.config.local_data_file)}"
            )
        else:
            logger.info("File already exists. Skipping download.")

       
    
    def extract_zip_file(self, zip_file_path: str):
        logger.info("Extracting zip file...")
        with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
            zip_ref.extractall(self.config.unzipped_data_dir)
        logger.info("Extraction completed.")
       


In [41]:
try:
    config_manager = ConfigurationManager()

    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = dataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file(
        zip_file_path=data_ingestion_config.local_data_file
    )
except Exception as e:
    raise e



>>> Trying to open: config/config.yaml
[2025-12-02 15:59:49,289: ERROR: Error reading YAML file: [Errno 2] No such file or directory: 'config/config.yaml']


FileNotFoundError: [Errno 2] No such file or directory: 'config/config.yaml'